In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_state_fractions_from_stats_batch(batch_folder, mode="both", max_step=None, denominator="all", title=None):
    """
    Plot % of gbm_n (necrotic) and/or gbm_q (quiescent) cells over time
    using your custom label map and color palette.

    Parameters:
    - batch_folder: path to folder with stats_*.pkl files
    - mode: "necrotic", "quiescent", or "both"
    - max_step: int or None — if set, only include steps <= max_step
    """

    label_map = {
        "10_1": "GBM + MSC 10:1",
        "3_1": "GBM + MSC 3:1",
        "1_1": "GBM + MSC 1:1",
        "1_3": "GBM + MSC 1:3",
        "2_1": "GBM + MSC 2:1",
        "5_1": "GBM + MSC 5:1",
        "20_1": "GBM + MSC 20:1",
        "GBM": "GBM Alone",
        "MSC": "MSC Alone"
    }

    color_palette = {
        "GBM": "#1f77b4",     # blue
        "1_1": "#2ca02c",     # green
        "MSC": "#ff7f0e",     # orange
        "1_3": "#d62728",     # red
        "3_1": "#9467bd",     # purple
        "2_1": "#8c564b",     # brown
        "5_1": "#e377c2",     # pink
        "10_1": "#7f7f7f",    # gray
        "20_1": "#bcbd22"     # yellow-green
    }

    stat_files = sorted([
        f for f in os.listdir(batch_folder)
        if f.startswith("stats_") and f.endswith(".pkl")
    ])

    plot_states = []
    if mode == "both":
        plot_states = [
            ("gbm_n", "Necrotic", "-"),
            ("gbm_q", "Quiescent", "--")
        ]
    elif mode == "necrotic":
        plot_states = [("gbm_n", "Necrotic", "-")]
    elif mode == "quiescent":
        plot_states = [("gbm_q", "Quiescent", "-")]


    if mode == "both":
        fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
    else:
        fig, ax = plt.subplots(figsize=(8, 5))

    for f in stat_files:
        key = f.replace("stats_", "").replace(".pkl", "")
        label = label_map.get(key, key)
        color = color_palette.get(key, "#333333")

        with open(os.path.join(batch_folder, f), "rb") as handle:
            data = pickle.load(handle)

        steps = np.array(data["steps"])
        num_cells = np.array(data["num_cells"])
        trends = data["state_trends"]

        if max_step is not None:
            mask = steps <= max_step
            steps = steps[mask]
            num_cells = num_cells[mask]

        for state_key, title, linestyle in plot_states:
            if state_key not in trends:
                continue
            # full arrays before masking
            full_steps = np.array(data["steps"])
            full_num_cells = np.array(data["num_cells"])

            # apply mask if needed
            if max_step is not None:
                mask = full_steps <= max_step
                steps = full_steps[mask]
                num_cells = full_num_cells[mask]
            else:
                steps = full_steps
                num_cells = full_num_cells

            state_counts = np.array(trends[state_key])
            if max_step is not None:
                state_counts = state_counts[mask]

            if denominator == "all":
                denom = num_cells
            else:  # "gbm+gsc"
                gbm_keys = [k for k in trends if k.startswith("gbm_") and len(trends[k]) == len(full_num_cells)]
                gsc_keys = [k for k in trends if k.startswith("gsc_") and len(trends[k]) == len(full_num_cells)]

                if gbm_keys:
                    gbm_total = np.stack([np.array(trends[k]) for k in gbm_keys]).sum(axis=0)
                else:
                    gbm_total = np.zeros_like(full_num_cells)

                if gsc_keys:
                    gsc_total = np.stack([np.array(trends[k]) for k in gsc_keys]).sum(axis=0)
                else:
                    gsc_total = np.zeros_like(full_num_cells)

                denom = gbm_total + gsc_total
                if max_step is not None:
                    denom = denom[mask]

            frac = np.divide(state_counts, denom, out=np.zeros_like(state_counts, dtype=float), where=denom > 0)



            if mode == "both":
                ax_idx = 0 if state_key == "gbm_n" else 1
                axes[ax_idx].plot(steps, frac, linestyle, label=label, color=color, alpha=0.9)
            else:
                ax.plot(steps, frac, linestyle, label=label, color=color, alpha=0.9)

    if mode == "both":
        for ax, (_, title, _) in zip(axes, plot_states):
            ax.set_xlabel("Step")
            ax.set_ylabel("Fraction of Cells")
            ax.set_title(f"{title} Cells Over Time")
            ax.legend()
        plt.tight_layout()
    else:
        ax.set_xlabel("Step")
        ax.set_ylabel("Fraction of Cells")
        ax.set_title(f"{plot_states[0][1]} Cells Over Time")
        ax.legend()
        plt.tight_layout()

    plt.show()


In [79]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import re
from glob import glob

def load_step(filepath):
    with h5py.File(filepath, "r") as f:
        cell_states = {int(k): f["cell_states"][k][()].decode("utf-8") for k in f["cell_states"]}
        cell_voxels = {int(k): f["cell_voxels"][k][()] for k in f["cell_voxels"]}
    return cell_states, cell_voxels

def compute_necrotic_radius(cell_states, cell_voxels, voxel_um=10.0):
    necrotic_voxels = []
    tumor_voxels = []

    for cid, state in cell_states.items():
        if cid not in cell_voxels:
            continue

        v = cell_voxels[cid]
        if isinstance(v, np.ndarray) and v.dtype.names:  # structured dtype
            voxels = np.stack([v[field] for field in v.dtype.names], axis=-1).astype(np.float32)
        else:
            voxels = np.array(v, dtype=np.float32)

        if state.startswith("gbm_") or state.startswith("gsc_"):
            tumor_voxels.extend(voxels)
            if state.endswith("_n"):
                necrotic_voxels.extend(voxels)



    if not necrotic_voxels or not tumor_voxels:
        return None  # skip if no data

    tumor_center = np.mean(tumor_voxels, axis=0)
    distances = np.linalg.norm(np.array(necrotic_voxels) - tumor_center, axis=1)
    distances_um = distances * voxel_um

    return {
        "mean": np.mean(distances_um),
        "max": np.max(distances_um),
        "p90": np.percentile(distances_um, 90)
    }

def plot_necrotic_core_radius(parent_folder, max_step=None):
    """
    For each condition in parent_folder, computes necrotic core radius
    (mean, max, 90th percentile) over time and plots all conditions in one figure.
    """
    # label + color maps (optional)
    label_map = {
        "10_1": "GBM + MSC 10:1", "3_1": "GBM + MSC 3:1", "1_1": "GBM + MSC 1:1",
        "1_3": "GBM + MSC 1:3", "2_1": "GBM + MSC 2:1", "5_1": "GBM + MSC 5:1",
        "20_1": "GBM + MSC 20:1", "GBM": "GBM Alone", "MSC": "MSC Alone"
    }

    color_palette = {
        "GBM": "#1f77b4", "1_1": "#2ca02c", "MSC": "#ff7f0e", "1_3": "#d62728",
        "3_1": "#9467bd", "2_1": "#8c564b", "5_1": "#e377c2",
        "10_1": "#7f7f7f", "20_1": "#bcbd22"
    }

    radius_data = {}  # {condition: {"step": [...], "mean": [...], "max": [...], "p90": [...]}}

    for condition in sorted(os.listdir(parent_folder)):
        cond_path = os.path.join(parent_folder, condition, "sim_output")
        if not os.path.isdir(cond_path):
            continue

        step_files = sorted(glob(os.path.join(cond_path, "step_*.jld2")))
        steps = []
        mean_r = []
        max_r = []
        p90_r = []

        for f in step_files:
            match = re.search(r"step_(\d+)", f)
            if not match:
                continue
            step = int(match.group(1))
            if max_step and step > max_step:
                continue

            cell_states, cell_voxels = load_step(f)
            radius = compute_necrotic_radius(cell_states, cell_voxels)
            if radius is None:
                continue

            steps.append(step)
            mean_r.append(radius["mean"])
            max_r.append(radius["max"])
            p90_r.append(radius["p90"])

        if steps:
            radius_data[condition] = {
                "step": steps, "mean": mean_r, "max": max_r, "p90": p90_r
            }

    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharex=True)

    for cond, data in radius_data.items():
        label = label_map.get(cond, cond)
        color = color_palette.get(cond, "#444444")
        steps = data["step"]

        axes[0].plot(steps, data["mean"], label=label, color=color)
        axes[1].plot(steps, data["max"], label=label, color=color)
        axes[2].plot(steps, data["p90"], label=label, color=color)

    axes[0].set_title("Mean Necrotic Radius (μm)")
    axes[1].set_title("Max Necrotic Radius (μm)")
    axes[2].set_title("90th Percentile Necrotic Radius (μm)")

    for ax in axes:
        ax.set_xlabel("Step")
        ax.set_ylabel("Radius (μm)")
        ax.legend()

    plt.tight_layout()
    plt.show()


In [87]:
def extract_voxel_distances_from_center(
    parent_folder,
    center=(35, 35, 35),
    out_folder="core_distances",
    max_step=None,
    overwrite=False
):
    """
    For each step in each condition, computes distances of necrotic (gbm_n)
    and quiescent (gbm_q) voxels from a fixed center point. Saves both in one .npz file.
    Resumable: skips steps that already exist unless overwrite=True.
    """
    os.makedirs(out_folder, exist_ok=True)

    for condition in sorted(os.listdir(parent_folder)):
        sim_path = os.path.join(parent_folder, condition, "sim_output")
        if not os.path.isdir(sim_path):
            continue

        print(f"⏳ Processing condition: {condition}")
        step_files = sorted(glob(os.path.join(sim_path, "step_*.jld2")))
        cond_out = os.path.join(out_folder, condition)
        os.makedirs(cond_out, exist_ok=True)

        for f in step_files:
            match = re.search(r"step_(\d+)", f)
            if not match:
                continue
            step = int(match.group(1))
            if max_step and step > max_step:
                continue

            out_file = os.path.join(cond_out, f"distances_step_{step:05d}.npz")
            if os.path.exists(out_file) and not overwrite:
                continue  # Skip already processed step

            try:
                with h5py.File(f, "r") as h5:
                    cell_states = {
                        int(k): h5["cell_states"][k][()].decode("utf-8")
                        for k in h5["cell_states"]
                    }
                    cell_voxels = {
                        int(k): h5["cell_voxels"][k][()]
                        for k in h5["cell_voxels"]
                    }
            except (KeyError, OSError, ValueError) as e:
                print(f"❌ Skipping corrupted file {f}: {e}")
                continue

            distances_n = []
            distances_q = []

            for cid, state in cell_states.items():
                if cid not in cell_voxels:
                    continue

                v = cell_voxels[cid]
                if isinstance(v, np.ndarray) and v.dtype.names:
                    vox = np.stack([v[f] for f in v.dtype.names], axis=-1).astype(np.float32)
                else:
                    vox = np.array(v, dtype=np.float32)

                dists = np.linalg.norm(vox - np.array(center, dtype=np.float32), axis=1)

                if state == "gbm_n":
                    distances_n.extend(dists)
                elif state == "gbm_q":
                    distances_q.extend(dists)

            np.savez_compressed(
                out_file,
                necrotic=np.array(distances_n),
                quiescent=np.array(distances_q)
            )

        print(f"✅ Finished {condition}")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
from glob import glob

def analyze_core_radius_distances(condition_folder, target="necrotic", label=None, verbose=True):
    """
    Loads .npz files for a single condition and plots:
    - Mean
    - Max
    - 90th percentile
    for the target core type: "necrotic" or "quiescent"
    """
    assert target in ["necrotic", "quiescent"], "Target must be 'necrotic' or 'quiescent'"

    npz_files = sorted(glob(os.path.join(condition_folder, "distances_step_*.npz")))
    if not npz_files:
        print(f" No .npz files found in {condition_folder}")
        return

    steps, means, maxes, p90s = [], [], [], []

    for f in npz_files:
        match = re.search(r"step_(\d+)", f)
        if not match:
            if verbose:
                print(f" Could not extract step number from {f}")
            continue
        step = int(match.group(1))

        try:
            data = np.load(f)
        except Exception as e:
            print(f" Failed to load {f}: {e}")
            continue

        if target not in data:
            if verbose:
                print(f" '{target}' not in {f} (available: {list(data.keys())})")
            continue

        dists = data[target]
        steps.append(step)

        if len(dists) == 0:
            if verbose:
                print(f"ℹ No {target} voxels in step {step} — inserting 0")
            means.append(0.0)
            maxes.append(0.0)
            p90s.append(0.0)
        else:
            means.append(np.mean(dists))
            maxes.append(np.max(dists))
            p90s.append(np.percentile(dists, 90))


    if not steps:
        print(f" No valid {target} data found in {condition_folder}")
        return

    # Sort by step
    steps, means, maxes, p90s = zip(*sorted(zip(steps, means, maxes, p90s)))

    label = label or os.path.basename(condition_folder)

    plt.figure(figsize=(8, 5))
    plt.plot(steps, means, label="Mean", lw=2)
    plt.plot(steps, maxes, label="Max", lw=2)
    plt.plot(steps, p90s, label="90th Percentile", lw=2)
    plt.title(f"{target.capitalize()} Core Radius Over Time — {label}")
    plt.xlabel("Step")
    plt.ylabel("Distance from Center (μm)")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
from glob import glob

def plot_mean_and_max_core_radii_all_conditions(parent_folder, target="necrotic", max_step=None, verbose=True):
    """
    For each condition folder in the parent folder, loads .npz distance files and
    plots two summary graphs:
        - Mean radius over time
        - Max radius over time
    for the specified core type: "necrotic" or "quiescent"
    """
    assert target in ["necrotic", "quiescent"], "Target must be 'necrotic' or 'quiescent'"

    condition_folders = sorted([
        os.path.join(parent_folder, d) for d in os.listdir(parent_folder)
        if os.path.isdir(os.path.join(parent_folder, d))
    ])

    if not condition_folders:
        print(f" No condition folders found in {parent_folder}")
        return

    all_means = {}
    all_maxes = {}

    for cond_folder in condition_folders:
        label = os.path.basename(cond_folder)
        npz_files = sorted(glob(os.path.join(cond_folder, "distances_step_*.npz")))

        if not npz_files:
            if verbose:
                print(f" No .npz files in {label}")
            continue

        steps, means, maxes = [], [], []

        for f in npz_files:
            match = re.search(r"step_(\d+)", f)
            if not match:
                continue
            step = int(match.group(1))
            if max_step is not None and step > max_step:
                continue

            try:
                data = np.load(f)
                if target not in data:
                    continue
                dists = data[target]
            except Exception as e:
                if verbose:
                    print(f" Failed to load {f}: {e}")
                continue

            steps.append(step)
            if len(dists) == 0:
                means.append(0.0)
                maxes.append(0.0)
            else:
                means.append(np.mean(dists))
                maxes.append(np.max(dists))

        if steps:
            steps, means, maxes = zip(*sorted(zip(steps, means, maxes)))
            voxel_size_um = 10  # adjust if different
            means = np.array(means) * voxel_size_um
            maxes = np.array(maxes) * voxel_size_um

            all_means[label] = (steps, means)
            all_maxes[label] = (steps, maxes)

    if not all_means:
        print(" No valid data found.")
        return

    # Plot Mean Radius
    plt.figure(figsize=(10, 5))
    for label, (steps, means) in all_means.items():
        plt.plot(steps, means, lw=2, label=label)
    plt.title(f"Mean {target.capitalize()} Radius Over Time")
    plt.xlabel("Step")
    plt.ylabel("Distance from Center (μm)")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

    # Plot Max Radius
    plt.figure(figsize=(10, 5))
    for label, (steps, maxes) in all_maxes.items():
        plt.plot(steps, maxes, lw=2, label=label)
    plt.title(f"Max {target.capitalize()} Radius Over Time")
    plt.xlabel("Step")
    plt.ylabel("Distance from Center (μm)")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
from glob import glob

def plot_core_growth_rate_all_conditions(
    parent_folder,
    target="necrotic",
    radius_type="mean",  # or "max"
    smooth_window=3,
    max_step=None,
    verbose=True
):
    """
    For each condition folder, computes and plots the growth rate of the core radius
    (mean or max) over time for the specified target state ("necrotic" or "quiescent").
    """
    assert target in ["necrotic", "quiescent"]
    assert radius_type in ["mean", "max"]

    condition_folders = sorted([
        os.path.join(parent_folder, d) for d in os.listdir(parent_folder)
        if os.path.isdir(os.path.join(parent_folder, d))
    ])

    all_growth_rates = {}

    for cond_folder in condition_folders:
        label = os.path.basename(cond_folder)
        npz_files = sorted(glob(os.path.join(cond_folder, "distances_step_*.npz")))

        steps, radii = [], []

        for f in npz_files:
            match = re.search(r"step_(\d+)", f)
            if not match:
                continue
            step = int(match.group(1))
            if max_step is not None and step > max_step:
                continue

            try:
                data = np.load(f)
                if target not in data:
                    continue
                dists = data[target]
            except Exception as e:
                if verbose:
                    print(f"Failed to load {f}: {e}")
                continue

            if len(dists) == 0:
                radius = 0.0
            else:
                radius = np.mean(dists) if radius_type == "mean" else np.max(dists)

            steps.append(step)
            radii.append(radius)

        if len(steps) < 2:
            continue

        # Sort and compute growth rate
        steps, radii = zip(*sorted(zip(steps, radii)))
        steps = np.array(steps)
        voxel_size_um = 10  # ← your case
        radii = np.array(radii) * voxel_size_um
        delta_r = np.diff(radii)
        delta_t = np.diff(steps)
        growth_rate = delta_r / delta_t
        mid_steps = (steps[:-1] + steps[1:]) / 2

        # Optional smoothing
        if smooth_window > 1:
            kernel = np.ones(smooth_window) / smooth_window
            growth_rate = np.convolve(growth_rate, kernel, mode="valid")
            mid_steps = mid_steps[:len(growth_rate)]

        all_growth_rates[label] = (mid_steps, growth_rate)

    # Plot
    plt.figure(figsize=(10, 5))
    for label, (x, y) in all_growth_rates.items():
        plt.plot(x, y, label=label, lw=2)

    plt.title(f"{radius_type.capitalize()} {target.capitalize()} Core Growth Rate")
    plt.xlabel("Step")
    plt.ylabel("Growth rate (μm / step)")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
